In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
cd /content/drive/My\ Drive/

/content/drive/My Drive


In [ ]:
!curl -O https://storage.googleapis.com/download.tensorflow.org/data/speech_commands_v0.02.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  3 2316M    3 84.2M    0     0  32.5M      0  0:01:11  0:00:02  0:01:09 32.5M^C


In [ ]:
!tar -xf speech_commands_v0.02.tar.gz

In [ ]:
!pwd

/content/drive/My Drive


In [ ]:
!python speech_commands/train.py --wanted_words=python

2020-09-16 09:53:06.327845: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-09-16 09:53:07.972196: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2299995000 Hz
2020-09-16 09:53:07.972524: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1dcabc0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-09-16 09:53:07.972560: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-09-16 09:53:07.974729: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-09-16 09:53:08.026596: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-09-16 09:53:08.027489: I tensorflow/compiler/xla/servic

In [ ]:
!tensorboard --logdir /tmp/retrain_logs

2020-08-19 09:50:29.613696: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.3.0 at http://localhost:6006/ (Press CTRL+C to quit)
^C


In [ ]:
!python speech_commands/freeze.py \
--start_checkpoint=/tmp/speech_commands_train/conv.ckpt-5200 \
--output_file=my_frozen_graph.pb

2020-09-16 10:50:03.055938: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-09-16 10:50:04.548933: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2299995000 Hz
2020-09-16 10:50:04.549229: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x2260bc0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-09-16 10:50:04.549271: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-09-16 10:50:04.551318: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-09-16 10:50:04.603651: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-09-16 10:50:04.604486: I tensorflow/compiler/xla/servic

In [ ]:
!python speech_commands/label_wav.py \
--graph=my_frozen_graph.pb \
--labels=conv_labels.txt \
--wav=speech_commands/test/out2.wav

2020-09-29 05:56:49.504320: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-09-29 05:56:51.151886: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-09-29 05:56:51.169343: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-09-29 05:56:51.170081: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla K80 computeCapability: 3.7
coreClock: 0.8235GHz coreCount: 13 deviceMemorySize: 11.17GiB deviceMemoryBandwidth: 223.96GiB/s
2020-09-29 05:56:51.170129: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-09-29 05:56:51.171602: I tensorflow/stream_executor/platform/defa

In [ ]:
import tensorflow as tf
from tensorflow.compat.v1.lite import TFLiteConverter

converter = TFLiteConverter.from_frozen_graph(
    'my_frozen_graph.pb', 
    input_arrays=['decoded_sample_data', 'decoded_sample_data:1'], 
    output_arrays=['labels_softmax'])

converter.allow_custom_ops=True
model = converter.convert()
open('model.tflite', 'wb').write(model)

1452192

In [ ]:
from google.colab import files

files.download('model.tflite')
files.download('conv_labels.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>